In [2]:
import os
from ms2query.utils import load_pickled_file

path_root = os.path.dirname(os.getcwd())
path_data = os.path.join(os.path.dirname(path_root), "data/gnps_24_11_2021/positive_mode/")


In [ ]:
training_scores, training_labels, validation_scores, validation_labels = load_pickled_file(os.path.join(path_data, "ms2query_library_files/training_data.pickle"))

### Check if the tanimoto score distribution is relatively equal

In [ ]:
plt.xlabel("tanimoto score")
plt.ylabel("Percentage (%)")
plt.hist(training_labels, weights = weights, rwidth = 0.9)
plt.show()
plt.xlabel("tanimoto score")
plt.ylabel("Percentage (%)")
plt.hist(validation_labels, weights = weights, rwidth = 0.9)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# train rf using optimised parameters from below
num_est = 250
rf = RandomForestRegressor(n_estimators = num_est, random_state = 42, max_depth = 5)
rf.fit(training_scores_top_100, training_labels_top_100)

# predict on train
rf_train_predictions = rf.predict(training_scores_top_100)
mse_train_rf = mean_squared_error(training_labels_top_100, rf_train_predictions)
print('Training MSE', mse_train_rf)

# predict on test
rf_predictions = rf.predict(validation_scores_top_100)
mse_rf = mean_squared_error(validation_labels_top_100, rf_predictions)
print('Validation MSE', mse_rf)

# get feature importances
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(training_scores_top_100.columns, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]